# Train BonDNet 

In this notebook, we show how to train the BonDNet graph neural network model for bond dissociation energy (BDE) prediction. We only show how to train on CPUs. See [train_bde_distributed.py](./) for a script for training on GPUs (a single GPU or distributed training on multiple GPUs). 

In [1]:
import torch
from torch.nn import MSELoss
from bondnet.data.dataset import ReactionNetworkDataset
from bondnet.data.dataloader import DataLoaderReactionNetwork
from bondnet.data.featurizer import AtomFeaturizerMinimum, BondAsNodeFeaturizerMinimum, GlobalFeaturizer, BondAsNodeFeaturizerFull
from bondnet.data.grapher import HeteroMoleculeGraph
from bondnet.data.dataset import train_validation_test_split
from bondnet.model.gated_reaction_network import GatedGCNReactionNetwork
from bondnet.scripts.create_label_file import read_input_files
from bondnet.model.metric import WeightedL1Loss
from bondnet.utils import seed_torch
from torchsummary import summary

/home/santiagovargas/anaconda3/envs/bondnet/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using backend: pytorch


In [2]:
print(torch.__version__)

1.6.0


## Dataset 

We work with a small dataset consisting of 200 BDEs for netural and charged molecules. The dataset is specified in three files:
- `molecules.sdf` This file contains all the molecules (both reactants and products) in the bond dissociation reactions. The molecules are specified in SDF format. 
- `molecule_attributes.yaml` This file contains extra molecular attributes (charges here) for molecules given in `molecules.sdf`. Some molecular attributes can be inferred from its SDF block, and they are overrode by the attributes specified in the `molecule_attributes.yaml` file.  
- `reactions.csv` This file list the bond dissociation reations formed by the molecules given in `molecules.sdf`. Each line lists the reactant, products, and BDE of a reaction. The reactant and products are specified by their index in `molecules.sdf`. 

See [here](./examples/train) for the three files used in this notebook. 

#### Grapher 

BondNet is graph neutral network model that takes atom features (e.g. atom type), bond features (e.g. whether a bond is in a ring), and global features (e.g. total charge) as input. We extract the features for a molecule using a grapher.

In [3]:
def get_grapher():
    atom_featurizer = AtomFeaturizerMinimum()
    #bond_featurizer = BondAsNodeFeaturizerMinimum()
    bond_featurizer = BondAsNodeFeaturizerFull()
    # our example dataset contains molecules of charges -1, 0, and 1
    global_featurizer = GlobalFeaturizer(allowed_charges=[-1, 0, 1])

    grapher = HeteroMoleculeGraph(atom_featurizer, bond_featurizer, global_featurizer)
    
    return grapher

#### Read dataset 

Let's now read the dataset and featurize the molecules using the above defined grapher. The dataset is split into a training set (80%), validation set (10%), and test set (10%). We will train our model using the training set, stop the training using the validation set, and report error on the test set. 

In [4]:
# seed random number generators 
seed_torch()

#mols, attrs, labels = read_input_files(
#    'examples/train/molecules.sdf', 
#    'examples/train/molecule_attributes.yaml', 
#    'examples/train/reactions.yaml', 
#)
path_mg_data = "/home/santiagovargas/Documents/Dataset/mg_dataset/"

mols, attrs, labels = read_input_files(
    #'examples/train/molecules.sdf', 
    #'examples/train/molecule_attributes.yaml', 
    #'examples/train/reactions.yaml', 
    path_mg_data + "mg_struct_bond_rgrn.sdf",
    path_mg_data + "mg_feature_bond_rgrn.yaml",
    path_mg_data + "mg_label_bond_rgrn.yaml",
)

dataset = ReactionNetworkDataset(
    grapher=get_grapher(),
    molecules=mols,
    labels=labels,
    extra_features=attrs
)

trainset, valset, testset = train_validation_test_split(dataset, validation=0.1, test=0.1)

# we train with a batch size of 100
train_loader = DataLoaderReactionNetwork(trainset, batch_size=100,shuffle=True)
val_loader = DataLoaderReactionNetwork(valset, batch_size=len(valset), shuffle=False)
test_loader = DataLoaderReactionNetwork(testset, batch_size=len(testset), shuffle=False)

RDKit WARNING: [11:23:05] Warning: molecule is tagged as 3D, but all Z coords are zero
[11:23:05] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [11:23:05] Warning: molecule is tagged as 3D, but all Z coords are zero
[11:23:05] Warning: molecule is tagged as 3D, but all Z coords are zero


KeyError: 'bond_mapping'

## Model 

We create the BonDNet model by instantiating the `GatedGCNReactionNetwork` class and providing the parameters defining the model structure. 
- `embedding_size` The size to unify the atom, bond, and global feature length.
- `gated_num_layers` Number of graph to graph module to learn molecular representation. 
- `gated_hidden_size` Hidden layer size in the graph to graph modules. 
- `gated_activation` Activation function appleid after the hidden layers in the graph to graph modules. 
- `fc_num_layers` Number of hidden layers of the fully connected network to map reaction feature to the BDE. The reaction feature is obtained as the differece of the features between the products and the reactant. 
- `fc_hidden_size` Size of the hidden layers. 
- `fc_activation` Activation function applied after the hidden layers. 

There are other arguments (e.g. residual connection, dropout ratio, batch norm) that can be specified to fine control the model. See the documentation of the `GatedGCNReactionNetwork` for more information.  

In [10]:
model = GatedGCNReactionNetwork(
    in_feats=dataset.feature_size,
    embedding_size=24,
    gated_num_layers=3,
    gated_hidden_size=[64, 64, 64],
    gated_activation="ReLU",
    fc_num_layers=2,
    fc_hidden_size=[128, 64],
    fc_activation='ReLU'
)

print(dict(model.named_parameters()))


{'embedding.linears.atom.weight': Parameter containing:
tensor([[ 0.0831, -0.2190, -0.0690,  0.0345, -0.0967, -0.0388, -0.0621,  0.2097,
          0.0379,  0.1082,  0.1509,  0.0549,  0.2306],
        [ 0.2483,  0.2188,  0.1781,  0.0779,  0.1232, -0.0681, -0.2124,  0.1531,
          0.2750,  0.0102,  0.1638, -0.0646,  0.2397],
        [ 0.1936, -0.0807, -0.2229, -0.2206, -0.0944,  0.0347, -0.1307, -0.0994,
          0.1047, -0.0211, -0.1326,  0.1818,  0.0391],
        [-0.1357,  0.2327, -0.0312,  0.1167, -0.2299,  0.1217,  0.0669, -0.0066,
          0.0269, -0.0870, -0.0997, -0.0719, -0.0856],
        [ 0.0003,  0.1721,  0.1651,  0.2614,  0.0189, -0.1066,  0.1660, -0.0364,
         -0.0264,  0.2160,  0.1613,  0.1629,  0.2548],
        [ 0.2476,  0.0394, -0.1063, -0.0036,  0.2051,  0.1314, -0.0531,  0.0628,
         -0.0707,  0.1013, -0.0571, -0.2314, -0.2139],
        [ 0.0151, -0.0129,  0.1979, -0.0546, -0.1075,  0.1587,  0.0027, -0.0334,
          0.2210,  0.2408,  0.0998, -0.1672,  0

## Train the model 

Before going to the main training loop, we define two functions: `train` and `evaluate` that will be used later. 

The `train` function optimizes the model parameters for an epoch. We note that our target BDEs are centered and then normalized by the standard deviation (done in the `ReactionNetworkDataset`.) So to measure the mean absolute error, we need to multiply the standard deviation back. This is acheived achieved by the `WeightedL1Loss` function passed as `metric_fn`.   

In [6]:
def train(optimizer, model, nodes, data_loader, loss_fn, metric_fn):

    model.train()

    epoch_loss = 0.0
    accuracy = 0.0
    count = 0.0

    for it, (batched_graph, label) in enumerate(data_loader):
        feats = {nt: batched_graph.nodes[nt].data["feat"] for nt in nodes}
        target = label["value"]
        stdev = label["scaler_stdev"]

        pred = model(batched_graph, feats, label["reaction"])
        pred = pred.view(-1)

        loss = loss_fn(pred, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() # here is the actual optimizer step

        epoch_loss += loss.detach().item()
        accuracy += metric_fn(pred, target, stdev).detach().item()
        count += len(target)
    
    epoch_loss /= it + 1
    accuracy /= count

    return epoch_loss, accuracy

The `evaluate` function computes the mean absolute error for the validation set (or test set).

In [7]:
def evaluate(model, nodes, data_loader, metric_fn):
    model.eval()

    with torch.no_grad():
        accuracy = 0.0
        count = 0.0

        for batched_graph, label in data_loader:
            feats = {nt: batched_graph.nodes[nt].data["feat"] for nt in nodes}
            target = label["value"]
            stdev = label["scaler_stdev"]

            pred = model(batched_graph, feats, label["reaction"])
            pred = pred.view(-1)

            accuracy += metric_fn(pred, target, stdev).detach().item()
            count += len(target)

    return accuracy / count

Now, we have all the ingredients to train the model. 

We optimize the model parameters by minimizing a mean squared error loss function using the `Adam` optimizer with a learning rate of `0.001`. Here we train the model for 20 epochs; save the best performing model that gets the smallest mean absolute error on the validation set; and finally test model performance on the test set. 

In [8]:
import time 

t1 = time.time()
# optimizer, loss function and metric function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = MSELoss(reduction="mean")
metric = WeightedL1Loss(reduction="sum")

feature_names = ["atom", "bond", "global"]
best = 1e10
num_epochs = 20

# main training loop
print("# Epoch     Loss         TrainAcc        ValAcc")
for epoch in range(num_epochs):

    # train on training set 
    loss, train_acc = train( optimizer, model, feature_names, train_loader, loss_func, metric)

    # evaluate on validation set
    val_acc = evaluate(model, feature_names, val_loader, metric)

    # save checkpoint for best performing model 
    is_best = val_acc < best
    if is_best:
        best = val_acc
        torch.save(model.state_dict(), 'checkpoint.pkl')
        
    print("{:5d}   {:12.6e}   {:12.6e}   {:12.6e}".format(epoch, loss, train_acc, val_acc))


# load best performing model and test it's performance on the test set
checkpoint = torch.load("checkpoint.pkl")
model.load_state_dict(checkpoint)
test_acc = evaluate(model, feature_names, test_loader, metric)

print("TestAcc: {:12.6e}".format(test_acc))
t2 = time.time()
print("Time to Training: {:5.1f} seconds".format(float(t2 - t1)))


# Epoch     Loss         TrainAcc        ValAcc
    0   8.582054e-01   2.501960e+00   1.776630e+00
    1   6.039712e-01   1.950059e+00   1.539107e+00
    2   4.469278e-01   1.758106e+00   1.345821e+00
    3   3.642534e-01   1.659739e+00   1.373917e+00
    4   2.783811e-01   1.397001e+00   1.475552e+00
    5   2.566877e-01   1.322044e+00   1.491818e+00
    6   2.037030e-01   1.198499e+00   1.323703e+00
    7   1.737967e-01   1.144645e+00   1.170846e+00
    8   1.565004e-01   1.092205e+00   1.122110e+00
    9   1.750356e-01   1.134772e+00   1.128524e+00
   10   1.209372e-01   9.319153e-01   1.088734e+00
   11   1.132107e-01   9.330008e-01   9.715240e-01
   12   9.905088e-02   8.184690e-01   9.910596e-01
   13   9.028393e-02   8.150457e-01   9.353060e-01
   14   8.694103e-02   8.235119e-01   1.020625e+00
   15   6.591935e-02   7.147754e-01   1.109250e+00
   16   7.335056e-02   7.355376e-01   1.059195e+00
   17   6.751134e-02   7.109309e-01   9.165952e-01
   18   6.396323e-02   7.041454e-0

In [11]:
# not working

#model.readout_layer.requires_grad_ = False
#for ind in range(len(model.gated_layers)):
#    model.gated_layers[ind].requires_grad_ = False
#for ind in range(len(model.fc_layers)):
#    model.fc_layers[ind].requires_grad_ = False
#print(dict(model.named_parameters()))

# works but messy 
# for ind, param in enumerate(model.parameters()):
#    print(ind)
#    param.requires_grad = True

model.gated_layers.requires_grad_(False)
model.fc_layers.requires_grad_(False)

import time 

t1 = time.time()
# optimizer, loss function and metric function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = MSELoss(reduction="mean")
metric = WeightedL1Loss(reduction="sum")

feature_names = ["atom", "bond", "global"]
best = 1e10
num_epochs = 20

# main training loop
print("# Epoch     Loss         TrainAcc        ValAcc")
for epoch in range(num_epochs):

    # train on training set 
    loss, train_acc = train( optimizer, model, feature_names, train_loader, loss_func, metric)

    # evaluate on validation set
    val_acc = evaluate(model, feature_names, val_loader, metric)

    # save checkpoint for best performing model 
    is_best = val_acc < best
    if is_best:
        best = val_acc
        torch.save(model.state_dict(), 'checkpoint.pkl')
        
    print("{:5d}   {:12.6e}   {:12.6e}   {:12.6e}".format(epoch, loss, train_acc, val_acc))


# load best performing model and test it's performance on the test set
checkpoint = torch.load("checkpoint.pkl")
model.load_state_dict(checkpoint)
test_acc = evaluate(model, feature_names, test_loader, metric)

print("TestAcc: {:12.6e}".format(test_acc))
t2 = time.time()

print("Time to Training: {:5.1f} seconds".format(float(t2 - t1)))



# Epoch     Loss         TrainAcc        ValAcc
    0   1.075364e+00   2.650825e+00   1.882989e+00
    1   9.821787e-01   2.625252e+00   1.880130e+00
    2   9.507621e-01   2.612707e+00   1.885765e+00
    3   8.983829e-01   2.590667e+00   1.883977e+00
    4   8.983957e-01   2.570117e+00   1.879380e+00
    5   9.359967e-01   2.542933e+00   1.871620e+00
    6   9.032701e-01   2.518162e+00   1.860771e+00
    7   9.331724e-01   2.504215e+00   1.846488e+00
    8   8.535580e-01   2.479704e+00   1.833602e+00
    9   8.570958e-01   2.450828e+00   1.818076e+00
   10   7.650453e-01   2.422336e+00   1.795764e+00
   11   8.498226e-01   2.391902e+00   1.768186e+00
   12   8.430975e-01   2.369736e+00   1.738051e+00
   13   7.846934e-01   2.338836e+00   1.708720e+00
   14   7.764476e-01   2.295667e+00   1.687622e+00
   15   7.120104e-01   2.265451e+00   1.666801e+00
   16   7.470441e-01   2.241836e+00   1.648616e+00
   17   6.573917e-01   2.199967e+00   1.609619e+00
   18   6.524919e-01   2.158915e+0

In [12]:


print(dict(model.named_parameters()))


{'embedding.linears.atom.weight': Parameter containing:
tensor([[ 4.9460e-02, -2.4917e-01, -8.0144e-02,  7.2956e-03, -1.2729e-01,
         -5.7689e-03, -8.2238e-02,  2.1951e-01,  3.3598e-02,  8.4139e-02,
          1.1528e-01,  7.2560e-02,  2.2278e-01],
        [ 2.1574e-01,  2.5080e-01,  1.4124e-01,  4.1507e-02,  1.1311e-01,
         -3.8082e-02, -1.8386e-01,  1.6025e-01,  2.6416e-01,  3.8437e-02,
          1.6790e-01, -8.0157e-02,  2.2799e-01],
        [ 2.2316e-01, -8.4115e-02, -1.8951e-01, -1.9814e-01, -1.1727e-01,
          6.4106e-02, -1.6739e-01, -1.2682e-01,  7.3353e-02, -5.8392e-04,
         -1.4186e-01,  2.0208e-01,  2.6837e-02],
        [-1.7118e-01,  2.0525e-01, -4.1438e-02,  8.0060e-02, -2.5566e-01,
          1.5491e-01,  2.8335e-02,  3.2526e-02,  3.8630e-02, -1.1949e-01,
         -1.2733e-01, -5.5469e-02, -6.9119e-02],
        [-2.5243e-02,  1.4605e-01,  1.3689e-01,  2.3643e-01,  4.6312e-02,
         -9.2213e-02,  1.3232e-01, -1.0464e-02, -2.2488e-02,  1.9374e-01,
        

In [15]:
# dict(model.named_parameters())
# print(dict(model.named_parameters())['embedding.linears.atom.weight'])
# for i in model.gated_layers:
#     i.requires_grad = False
# for i in model.fc_layers:
#     i.requires_grad = False
# model.parameters['gated_layers']
# model.fc_layers[-1]['weight'].requires_grad = False
# print(dict(model.fc_layers[-1].named_parameters())['weight'])
# print(dict(model.fc_layers.named_parameters()))
# fc_layers.4.bias
#for i in dict(model.named_parameters()):
print(model.parameters())


<generator object Module.parameters at 0x7f6a82d5e408>


In [3]:
import pandas as pd 
mg_file_ref = "/home/santiagovargas/Documents/Dataset/20210721_madeira_g2.json" 
libe_file_ref = "/home/santiagovargas/Documents/Dataset/libe.json"
mg_df = pd.read_json(mg_file_ref)
libe_df = pd.read_json(libe_file_ref)

In [9]:
libe_df.columns

Index(['molecule_id', 'bonds', 'charge', 'chemical_system', 'composition',
       'elements', 'formula_alphabetical', 'molecule', 'molecule_graph',
       'number_atoms', 'number_elements', 'partial_charges', 'partial_spins',
       'point_group', 'species', 'spin_multiplicity', 'thermo', 'vibration',
       'xyz'],
      dtype='object')

In [8]:
mg_df.columns

Index(['molecule_id', 'bonds', 'charge', 'chemical_system', 'composition',
       'elements', 'formula_alphabetical', 'molecule', 'molecule_graph',
       'number_atoms', 'number_elements', 'partial_charges', 'partial_spins',
       'point_group', 'species', 'spin_multiplicity', 'thermo', 'vibration',
       'xyz'],
      dtype='object')